In [2]:
import os
import streamlit as st
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
groq_api_key = os.getenv('GROQ_API_KEY')
os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")

In [5]:
st.title("Gemma Model Document Q&A")

DeltaGenerator()

In [8]:
llm=ChatGroq(groq_api_key=groq_api_key, model="Gemma-7b-it")

In [9]:
prompt=ChatPromptTemplate.from_template(
  """
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}


"""  
)

In [10]:
def vector_embedding():
    if "vectors" not in st.session_state:
        st.session_state.embeddings=GoogleGenerativeAIEmbeddings(model="models/emnbedding-001")
        st.session_state.loader = PyPDFDirectoryLoader("./us_census")
        st.session_state.docs=st.session_state.loader.load()
        st.session_state.text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        st.session_state.final_documents=st.session_state.text_splitter.split_documents(st.session_state.docs)
        st.session_state.vectors=FAISS.from_documents(st.session_state.final_documents, st.session_state.embeddings)


In [11]:
prompt1=st.text_input("What you want to ask from the documents?")

2024-05-23 00:20:38.752 Session state does not function when running a script without `streamlit run`


In [12]:
if st.button('Create Vector Store'):
    vector_embedding()
    st.write("Vector store DB IS Ready")


In [13]:
if prompt1:
    document_chain=create_stuff_documents_chain(llm, prompt=prompt)
    retriever=st.session_state.vectors.as_retriever()
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    start=time.process_time()
    response = retrieval_chain.invoke({"input":prompt1})
    st.write(response['answer'])

    with st.expander("Document Similarity Search"):
        for i,doc in enumerate(response["context"]):
            st.write(doc.page_content)
            st.write("----------------------")

In [14]:
!streamlit run

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Missing argument 'TARGET'.


In [15]:
!streamlit run app.py

^C
